In [1]:
from flask import Flask, render_template, request, redirect, url_for
import numpy as np
import PIL
import cv2
import matplotlib.pyplot as plt
import face_recognition as face_r
import pymysql
import datetime

In [2]:
app = Flask(__name__)

In [3]:
@app.route('/')
def home():
    return render_template('index.html')

In [4]:
@app.route('/register/', methods = ["POST", "GET"])
def register():
    if request.method == "POST":
        conn = pymysql.connect(host = 'localhost', user = 'root', database = 'Project', password = 'L0srudra333@')
        cur_1 = conn.cursor()
        roll_no = request.form["roll_no"]
        f_name = request.form["fname"]
        img = cv2.imread(request.form["img"])
        face_enc = face_r.face_encodings(img)
        
        if(len(face_enc) == 0):
            num_row, num_col = img.shape[:2]
            rotation_matrix = cv2.getRotationMatrix2D((num_col/2, num_row/2), -90, 1)
            rotated_img = cv2.warpAffine(img, rotation_matrix, (num_col, num_row))
            face_enc = face_r.face_encodings(rotated_img)
            if(len(face_enc)==0):
                num_row, num_col = rotated_img.shape[:2]
                rotation_matrix = cv2.getRotationMatrix2D((num_col/2, num_row/2), -90, 1)
                rotated_img = cv2.warpAffine(rotated_img, rotation_matrix, (num_col, num_row))
                face_enc = face_r.face_encodings(rotated_img)
                
                if(len(face_enc) == 0):
                    num_row, num_col = rotated_img.shape[:2]
                    rotation_matrix = cv2.getRotationMatrix2D((num_col/2, num_row/2), -90, 1)
                    rotated_img = cv2.warpAffine(rotated_img, rotation_matrix, (num_col, num_row))
                    face_enc = face_r.face_encodings(rotated_img)
        
        if(len(face_enc) == 1):
            qry = 'insert into Student_info(name, roll_no, face_enc) values(%s, %s, %s)'
            param = (str(f_name), int(roll_no), str(face_enc))
            cur_1.execute(qry, param)
            conn.commit()
            return redirect(url_for('home'))
        else:
            return "registration failed !! TRY AGIAN :):)"
        
    else:
        print('Registration start.............!!')
        return render_template('login.html')

In [5]:
@app.route('/attendance/', methods = ["POST", "GET"])
def Attendance():
    if request.method == "POST":
        conn = pymysql.connect(host = 'localhost', user = 'root', database = 'Project', password = 'L0srudra333@')
        cur = conn.cursor()
        f_name = request.form['fname']
        roll_no = request.form['roll_no']
        img = cv2.imread(request.form['img'])
        
        face_loc = face_r.face_locations(img, model = 'cnn')
        top, right, bottom, left = face_loc[0]
        img = img[top:bottom, left:right]
        face_enc = face_r.face_encodings(img)
        
        qry = 'select face_enc from Student_info where roll_no = %s'
        param = (roll_no)
        cur.execute(qry, param)
        face_enc_ = cur.fetchall()[0][0]
        
        face_e = []
        face_enc_= face_enc_[8:-3]
        for ii in face_enc_.split(',\n'):
            for i in ii.split(','):
                face_e.append(float(i))
                
            
        face_e = np.array(face_e, dtype = 'float64')    
        result = face_r.compare_faces(face_e, face_enc)[0]
        now = datetime.date.today()
        conn = pymysql.connect(host = 'localhost', user = 'root', database = 'Project', password = 'L0srudra333@')
        cur = conn.cursor()
        
        if(result):
            qry = f'update Student_attendance set `00{roll_no}` = 1 where date = "{now}"'
            cur.execute(qry)
            return f'Dear {f_name}: your attendance has been recorded'
        
        else:
            return 'Try Again :)'
     
        
    else:
        print("varifying.........:)")
        return render_template('attendance.html')

In [ ]:
if(__name__ == "__main__"):
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2020 01:27:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 01:27:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Nov/2020 01:27:43] "GET /attendance/ HTTP/1.1" 200 -


varifying.........:)


127.0.0.1 - - [22/Nov/2020 01:27:49] "GET /register/ HTTP/1.1" 200 -


Registration start.............!!


127.0.0.1 - - [22/Nov/2020 01:29:43] "GET /login/ HTTP/1.1" 404 -
127.0.0.1 - - [22/Nov/2020 01:29:43] "GET /favicon.ico HTTP/1.1" 404 -
